# load packages

In [ ]:
library(tidyverse)
library(bigrquery)

# Load v8 data

## thyroid meds

In [ ]:
# This query represents dataset "TSH pheno v8" for domain "drug" and was generated for All of Us Controlled Tier Dataset v8
dataset_17403006_drug_sql <- paste("
    SELECT
        d_exposure.person_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_exposure.drug_exposure_start_datetime 
    FROM
        ( SELECT
            * 
        FROM
            `drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN (SELECT
                    DISTINCT ca.descendant_id 
                FROM
                    `cb_criteria_ancestor` ca 
                JOIN
                    (SELECT
                        DISTINCT c.concept_id       
                    FROM
                        `cb_criteria` c       
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id             
                        FROM
                            `cb_criteria` cr             
                        WHERE
                            concept_id IN (1501309, 1501700, 1505346, 1590418, 21602751, 21602752, 21602753, 21605503, 21605504)             
                            AND full_text LIKE '%_rank1]%'       ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) b 
                        ON (ca.ancestor_id = b.concept_id)))  
                    AND (d_exposure.PERSON_ID IN (SELECT
                        distinct person_id  
                FROM
                    `cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (3023399, 37052205, 3006499, 3017044, 3044958, 4197602, 3045302, 3019762, 3032069, 3016806, 3038369, 3001019, 3016735, 3009201, 3019170, 4246879, 3015559, 40760528, 3009023, 3006939, 3027700)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (35206909, 35206912, 35206914, 35206911, 35206913, 35206910, 1567993)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria 
                    UNION
                    DISTINCT SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (1567895, 1567893, 35206304)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria 
                    UNION
                    DISTINCT SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (44825214, 44829873, 44822938, 44827619, 44833373, 44820687, 44819497, 44828784, 44825267)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) )
                )
            ) d_exposure 
        LEFT JOIN
            `concept` d_standard_concept 
                ON d_exposure.drug_concept_id = d_standard_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
drug_17403006_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "drug_17403006",
  "drug_17403006_*.csv")
message(str_glue('The data will be written to {drug_17403006_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_17403006_drug_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  drug_17403006_path,
  destination_format = "CSV")



In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {drug_17403006_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
meds_raw <- read_bq_export_from_workspace_bucket(drug_17403006_path)

dim(meds_raw)

head(meds_raw, 5)

## pregnancy observations

In [ ]:
# This query represents dataset "TSH pheno v8" for domain "observation" and was generated for All of Us Controlled Tier Dataset v8
dataset_17403006_observation_sql <- paste("
    SELECT
        observation.person_id,
        observation.observation_datetime,
        observation.observation_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `observation` observation 
        WHERE
            (
                observation_source_concept_id IN (37200953, 37200954, 37200955, 44819867, 44819897, 44819898, 44821027, 44821038, 44821043, 44821045, 44821050, 44822088, 44822100, 44822113, 44822124, 44823266, 44823290, 44825609, 44826802, 44827960, 44829139, 44829145, 44830268, 44831391, 44831401, 44831406, 44832537, 44832553, 44832554, 44832564, 44833747, 44833757, 44834896, 44836102, 44836107, 44836108, 45534164, 45534165, 45534167, 45534168, 45543911, 45543912, 45548692, 45548693, 45548694, 45548696, 45553473, 45553474, 45558192, 45558193, 45558194, 45558195, 45558196, 45558199, 45563043, 45563044, 45563045, 45563046, 45567884, 45572757, 45572758, 45572759, 45572760, 45577553, 45577554, 45577555, 45582439, 45582440, 45582441, 45587273, 45587274, 45587275, 45587276, 45587277, 45587279, 45592175, 45592176, 45592177, 45592178, 45592179, 45592180, 45592181, 45592182, 45596912, 45596913, 45601773, 45601774, 45601775, 45601776, 45606534, 45606535, 45606536)
            )  
            AND (
                observation.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (3023399, 37052205, 3006499, 3017044, 3044958, 4197602, 3045302, 3019762, 3032069, 3016806, 3038369, 3001019, 3016735, 3009201, 3019170, 4246879, 3015559, 40760528, 3009023, 3006939, 3027700)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (35206909, 35206912, 35206914, 35206911, 35206913, 35206910, 1567993)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria 
                    UNION
                    DISTINCT SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (1567895, 1567893, 35206304)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria 
                    UNION
                    DISTINCT SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (44825214, 44829873, 44822938, 44827619, 44833373, 44820687, 44819497, 44828784, 44825267)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) )
                )
            ) observation", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
observation_17403006_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "observation_17403006",
  "observation_17403006_*.csv")
message(str_glue('The data will be written to {observation_17403006_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_17403006_observation_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  observation_17403006_path,
  destination_format = "CSV")



In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {observation_17403006_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(observation_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
preg_observation_raw <- read_bq_export_from_workspace_bucket(observation_17403006_path)

dim(preg_observation_raw)

head(preg_observation_raw, 5)

## pregnancy conditions

In [ ]:
# This query represents dataset "TSH pheno v8" for domain "condition" and was generated for All of Us Controlled Tier Dataset v8
dataset_17403006_condition_sql <- paste("
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `condition_occurrence` c_occurrence 
        WHERE
            (
                condition_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (1326728, 1326729, 1326730, 1326731, 1326732, 1326733, 1326734, 1326735, 1326736, 1326737, 1326738, 1326739, 1571602, 1571603, 1571605, 1571614, 1571633, 35209513, 35209514, 35209515, 35209516, 35209517, 35209518, 35209519, 35209520, 35209521, 35209522, 35209524, 35209525, 35209526, 35209527, 35209528, 35209529, 35209530, 35209531, 35209532, 35209537, 35209539, 35209540, 35209541, 35209542, 35209543, 35209544, 35209545, 35209546, 35209547, 35209548, 37200942, 37200943, 37200944, 37200945, 37200946, 37200948, 37200949, 37200950, 37200951, 37200953, 37200954, 37200955, 37200956, 44819846, 44819847, 44819848, 44819849, 44819850, 44819851, 44819852, 44819853, 44819854, 44819856, 44819858, 44819859, 44819860, 44819861, 44819863, 44819864, 44819866, 44819867, 44819868, 44819869, 44819871, 44819872, 44819874, 44819876, 44819878, 44819881, 44819882, 44819883, 44819884, 44819885, 44819886, 44819887, 44819888, 44819890, 44819891, 44819892, 44819893, 44819894,
 44819895, 44819896, 44819898, 44819900, 44819902, 44819904, 44819906, 44819909, 44819910, 44819912, 44819913, 44819914, 44819916, 44819918, 44819920, 44819921, 44819922, 44819923, 44819924, 44819925, 44819928, 44819929, 44819931, 44819932, 44821001, 44821002, 44821003, 44821004, 44821006, 44821007, 44821008, 44821014, 44821016, 44821017, 44821018, 44821019, 44821020, 44821021, 44821023, 44821027, 44821029, 44821031, 44821032, 44821033, 44821034, 44821037, 44821039, 44821041, 44821043, 44821044, 44821045, 44821047, 44821048, 44821049, 44821052, 44821053, 44821055, 44821056, 44821057, 44821058, 44821059, 44821065, 44821067, 44821070, 44821073, 44821074, 44821075, 44821076, 44821077, 44821078, 44821079, 44821080, 44821081, 44821082, 44822064, 44822066, 44822067, 44822068, 44822079, 44822080, 44822081, 44822082, 44822083, 44822084, 44822085, 44822087, 44822088, 44822090, 44822091, 44822093, 44822094, 44822097, 44822098, 44822099, 44822100, 44822101, 44822102, 44822103, 44822104, 44822105,
 44822107, 44822108, 44822109, 44822110, 44822111, 44822112, 44822113, 44822114, 44822116, 44822117, 44822118, 44822119, 44822120, 44822121, 44822122, 44822124, 44822125, 44822126, 44822127, 44822128, 44822129, 44822130, 44822131, 44822134, 44822135, 44822136, 44822137, 44822140, 44822141, 44822143, 44822144, 44822150, 44822153, 44822156, 44822158, 44822159, 44823216, 44823217, 44823218, 44823221, 44823227, 44823228, 44823229, 44823230, 44823231, 44823235, 44823236, 44823237, 44823238, 44823239, 44823241, 44823243, 44823245, 44823246, 44823247, 44823248, 44823249, 44823250, 44823255, 44823258, 44823259, 44823260, 44823262, 44823263, 44823264, 44823265, 44823266, 44823267, 44823268, 44823269, 44823270, 44823272, 44823273, 44823274, 44823275, 44823278, 44823279, 44823280, 44823281, 44823282, 44823283, 44823284, 44823285, 44823286, 44823287, 44823289, 44823291, 44823293, 44823295, 44823296, 44823299, 44823301, 44823302, 44823304, 44823305, 44823307, 44823310, 44823311, 44823312, 44823313,
 44823314, 44824369, 44824370, 44824371, 44824372, 44824374, 44824384, 44824385, 44824386, 44824388, 44824389, 44824390, 44824391, 44824392, 44824393, 44824394, 44824395, 44824396, 44824397, 44824398, 44824399, 44824402, 44824403, 44824404, 44824405, 44824406, 44824407, 44824408, 44824409, 44824411, 44824413, 44824414, 44824415, 44824416, 44824417, 44824418, 44824419, 44824420, 44824421, 44824422, 44824423, 44824425, 44824427, 44824428, 44824429, 44824430, 44824431, 44824432, 44824433, 44824434, 44824435, 44824437, 44824438, 44824440, 44824441, 44824443, 44824444, 44824445, 44824448, 44824449, 44824453, 44824456, 44824459, 44824460, 44824461, 44824463, 44824465, 44824467, 44825572, 44825582, 44825585, 44825586, 44825587, 44825588, 44825589, 44825590, 44825591, 44825592, 44825593, 44825594, 44825595, 44825596, 44825599, 44825600, 44825601, 44825602, 44825603, 44825604, 44825605, 44825606, 44825607, 44825608, 44825609, 44825610, 44825611, 44825613, 44825615, 44825618, 44825619, 44825620,
 44825621, 44825622, 44825623, 44825624, 44825626, 44825629, 44825631, 44825633, 44825634, 44825635, 44825636, 44825638, 44825639, 44825640, 44825642, 44825644, 44825645, 44825646, 44826756, 44826757, 44826758, 44826760, 44826761, 44826762, 44826769, 44826770, 44826771, 44826773, 44826774, 44826775, 44826776, 44826777, 44826778, 44826779, 44826780, 44826784, 44826786, 44826787, 44826789, 44826791, 44826792, 44826796, 44826797, 44826798, 44826800, 44826801, 44826802, 44826804, 44826805, 44826806, 44826807, 44826808, 44826809, 44826813, 44826814, 44826818, 44826819, 44826820, 44826821, 44826822, 44826823, 44826824, 44826825, 44826827, 44826828, 44826829, 44826830, 44826831, 44826834, 44826836, 44826840, 44827917, 44827923, 44827929, 44827930, 44827931, 44827932, 44827933, 44827934, 44827935, 44827936, 44827937, 44827938, 44827939, 44827940, 44827941, 44827942, 44827943, 44827944, 44827945, 44827946, 44827947, 44827948, 44827949, 44827950, 44827951, 44827952, 44827954, 44827955, 44827956,
 44827957, 44827958, 44827959, 44827960, 44827961, 44827962, 44827963, 44827964, 44827965, 44827966, 44827967, 44827968, 44827969, 44827970, 44827971, 44827972, 44827974, 44827975, 44827977, 44827978, 44827979, 44827980, 44827981, 44827983, 44827985, 44827986, 44827988, 44827990, 44827991, 44827992, 44827993, 44827995, 44827996, 44827997, 44827998, 44829096, 44829097, 44829098, 44829099, 44829100, 44829101, 44829102, 44829103, 44829104, 44829105, 44829106, 44829108, 44829109, 44829110, 44829114, 44829115, 44829116, 44829117, 44829118, 44829120, 44829121, 44829122, 44829125, 44829126, 44829127, 44829128, 44829130, 44829131, 44829132, 44829133, 44829135, 44829138, 44829139, 44829141, 44829142, 44829145, 44829146, 44829147, 44829148, 44829149, 44829150, 44829152, 44829154, 44829155, 44829156, 44829158, 44829159, 44829161, 44829162, 44829163, 44829164, 44829167, 44829168, 44829169, 44829170, 44829171, 44829173, 44830199, 44830200, 44830205, 44830206, 44830207, 44830208, 44830209, 44830210,
 44830211, 44830212, 44830213, 44830214, 44830215, 44830216, 44830217, 44830218, 44830219, 44830220, 44830221, 44830222, 44830223, 44830225, 44830226, 44830229, 44830230, 44830232, 44830233, 44830234, 44830235, 44830236, 44830238, 44830239, 44830240, 44830241, 44830242, 44830243, 44830245, 44830248, 44830250, 44830252, 44830253, 44830255, 44830258, 44830261, 44830262, 44830263, 44830264, 44830265, 44830266, 44830267, 44830269, 44830274, 44830276, 44830282, 44830283, 44830284, 44830287, 44830288, 44830290, 44830291, 44830292, 44830293, 44831364, 44831371, 44831372, 44831373, 44831374, 44831375, 44831376, 44831377, 44831378, 44831379, 44831380, 44831381, 44831383, 44831384, 44831385, 44831386, 44831387, 44831388, 44831389, 44831390, 44831391, 44831392, 44831393, 44831394, 44831395, 44831396, 44831397, 44831398, 44831399, 44831400, 44831401, 44831403, 44831404, 44831405, 44831406, 44831408, 44831409, 44831410, 44831411, 44831412, 44831413, 44831414, 44831415, 44831416, 44831417, 44831419,
 44831422, 44831423, 44831424, 44831425, 44831426, 44831428, 44831431, 44831433, 44831434, 44831435, 44831436, 44831437, 44831438, 44831440, 44831441, 44831442, 44831445, 44831446, 44831447, 44831449, 44831450, 44831452, 44831454, 44832507, 44832509, 44832513, 44832518, 44832519, 44832520, 44832521, 44832522, 44832523, 44832524, 44832525, 44832526, 44832528, 44832529, 44832531, 44832532, 44832533, 44832535, 44832536, 44832537, 44832538, 44832539, 44832541, 44832542, 44832543, 44832544, 44832545, 44832546, 44832547, 44832548, 44832549, 44832550, 44832551, 44832553, 44832554, 44832556, 44832557, 44832558, 44832559, 44832561, 44832562, 44832564, 44832566, 44832568, 44832570, 44832571, 44832572, 44832573, 44832574, 44832577, 44832579, 44832581, 44832582, 44832585, 44832587, 44832588, 44832591, 44832593, 44832594, 44832595, 44832596, 44832597, 44833693, 44833694, 44833695, 44833696, 44833697, 44833698, 44833712, 44833713, 44833714, 44833715, 44833717, 44833718, 44833719, 44833721, 44833722,
 44833723, 44833724, 44833725, 44833726, 44833727, 44833728, 44833730, 44833733, 44833734, 44833736, 44833737, 44833738, 44833739, 44833740, 44833741, 44833742, 44833743, 44833744, 44833745, 44833747, 44833749, 44833751, 44833752, 44833753, 44833754, 44833755, 44833756, 44833757, 44833758, 44833759, 44833761, 44833763, 44833764, 44833768, 44833770, 44833771, 44833772, 44833773, 44833776, 44833779, 44833780, 44833781, 44833782, 44834860, 44834867, 44834872, 44834873, 44834874, 44834875, 44834876, 44834877, 44834878, 44834879, 44834881, 44834882, 44834883, 44834884, 44834885, 44834886, 44834888, 44834890, 44834891, 44834893, 44834894, 44834895, 44834897, 44834898, 44834900, 44834901, 44834904, 44834905, 44834906, 44834908, 44834909, 44834911, 44834912, 44834913, 44834914, 44834915, 44834919, 44834920, 44834921, 44834922, 44834923, 44834924, 44834927, 44834928, 44834929, 44834931, 44834933, 44834934, 44834937, 44834942, 44834943, 44834946, 44834947, 44836063, 44836064, 44836065, 44836066,
 44836070, 44836071, 44836072, 44836073, 44836074, 44836075, 44836076, 44836077, 44836078, 44836079, 44836081, 44836084, 44836085, 44836086, 44836087, 44836088, 44836089, 44836090, 44836091, 44836092, 44836093, 44836094, 44836096, 44836097, 44836099, 44836100, 44836102, 44836104, 44836106, 44836109, 44836111, 44836112, 44836113, 44836114, 44836118, 44836120, 44836121, 44836122, 44836125, 44836126, 44836127, 44836128, 44836129, 44836132, 44836134, 44836135, 44836137, 44836138, 44836139, 44836141, 44836142, 44836143, 44836144, 44836145, 44837213, 44837214, 44837215, 44837226, 44837228, 44837229, 44837231, 44837232, 44837233, 44837234, 44837235, 44837236, 44837237, 44837238, 44837239, 44837240, 44837241, 44837242, 44837243, 44837244, 44837245, 44837246, 44837247, 44837248, 44837249, 44837250, 44837251, 44837252, 44837253, 44837254, 44837255, 44837256, 44837257, 44837258, 44837259, 44837260, 44837261, 44837262, 44837263, 44837267, 44837268, 44837271, 44837272, 44837273, 44837274, 44837276,
 44837277, 44837280, 44837281, 44837282, 44837283, 44837284, 44837285, 44837286, 44837288, 44837292, 44837293, 44837294, 44837296, 44837297, 44837298, 44837299, 44837300, 44837301, 45534162, 45534163, 45534164, 45534165, 45534167, 45534168, 45543911, 45543912, 45548688, 45548689, 45548692, 45548693, 45548694, 45548696, 45553466, 45553470, 45553473, 45553474, 45558184, 45558186, 45558192, 45558193, 45558195, 45558196, 45558199, 45563038, 45563039, 45563043, 45563044, 45563045, 45563046, 45567880, 45567884, 45572746, 45572748, 45572749, 45572753, 45572757, 45572758, 45572759, 45572760, 45577553, 45577554, 45577555, 45582435, 45582439, 45582440, 45582441, 45587273, 45587274, 45587275, 45587276, 45587277, 45587279, 45592169, 45592171, 45592173, 45592175, 45592176, 45592178, 45592179, 45592180, 45592181, 45592182, 45596897, 45596898, 45596899, 45596911, 45596912, 45596913, 45601765, 45601772, 45601773, 45601774, 45601775, 45601776, 45606534, 45606535, 45606536)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (3023399, 37052205, 3006499, 3017044, 3044958, 4197602, 3045302, 3019762, 3032069, 3016806, 3038369, 3001019, 3016735, 3009201, 3019170, 4246879, 3015559, 40760528, 3009023, 3006939, 3027700)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (35206909, 35206912, 35206914, 35206911, 35206913, 35206910, 1567993)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria 
                    UNION
                    DISTINCT SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (1567895, 1567893, 35206304)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria 
                    UNION
                    DISTINCT SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (44825214, 44829873, 44822938, 44827619, 44833373, 44820687, 44819497, 44828784, 44825267)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) )
                )
            ) c_occurrence", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
condition_17403006_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "condition_17403006",
  "condition_17403006_*.csv")
message(str_glue('The data will be written to {condition_17403006_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_17403006_condition_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  condition_17403006_path,
  destination_format = "CSV")



In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {condition_17403006_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(condition_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
preg_condition_raw <- read_bq_export_from_workspace_bucket(condition_17403006_path)

dim(preg_condition_raw)

head(preg_condition_raw, 5)

## TSH labs

In [ ]:
# This query represents dataset "TSH pheno v8" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v8
dataset_17403006_measurement_sql <- paste("
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (3000551, 3001019, 3006499, 3006939, 3008598, 3009023, 3009201, 3010340, 3015559, 3016735, 3016806, 3016991, 3017044, 3019170, 3019762, 3021717, 3023399, 3025547, 3026925, 3027238, 3027700, 3032069, 3038369, 3044958, 3045302, 37034510, 37050150, 37052205, 37053647, 40760528, 40796118, 4190803, 4197602, 4246879, 42529233, 42530171)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1) 
                OR  measurement_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (2212591, 2212593, 2212603)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )  
            AND (
                measurement.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (3023399, 37052205, 3006499, 3017044, 3044958, 4197602, 3045302, 3019762, 3032069, 3016806, 3038369, 3001019, 3016735, 3009201, 3019170, 4246879, 3015559, 40760528, 3009023, 3006939, 3027700)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (35206909, 35206912, 35206914, 35206911, 35206913, 35206910, 1567993)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria 
                    UNION
                    DISTINCT SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (1567895, 1567893, 35206304)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria 
                    UNION
                    DISTINCT SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (44825214, 44829873, 44822938, 44827619, 44833373, 44820687, 44819497, 44828784, 44825267)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) )
                )
            ) measurement 
        LEFT JOIN
            `concept` m_standard_concept 
                ON measurement.measurement_concept_id = m_standard_concept.concept_id 
        LEFT JOIN
            `concept` m_type 
                ON measurement.measurement_type_concept_id = m_type.concept_id 
        LEFT JOIN
            `concept` m_operator 
                ON measurement.operator_concept_id = m_operator.concept_id 
        LEFT JOIN
            `concept` m_value 
                ON measurement.value_as_concept_id = m_value.concept_id 
        LEFT JOIN
            `concept` m_unit 
                ON measurement.unit_concept_id = m_unit.concept_id 
        LEFT JOIn
            `visit_occurrence` v 
                ON measurement.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `concept` m_visit 
                ON v.visit_concept_id = m_visit.concept_id 
        LEFT JOIN
            `concept` m_source_concept 
                ON measurement.measurement_source_concept_id = m_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
measurement_17403006_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "measurement_17403006",
  "measurement_17403006_*.csv")
message(str_glue('The data will be written to {measurement_17403006_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_17403006_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_17403006_path,
  destination_format = "CSV")



In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_17403006_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), measurement_type_concept_name = col_character(), operator_concept_name = col_character(), value_as_concept_name = col_character(), unit_concept_name = col_character(), visit_occurrence_concept_name = col_character(), measurement_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), unit_source_value = col_character(), value_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
labs_raw <- read_bq_export_from_workspace_bucket(measurement_17403006_path)

dim(labs_raw)

head(labs_raw, 5)

## demo

In [ ]:
# This query represents dataset "TSH pheno v8" for domain "person" and was generated for All of Us Controlled Tier Dataset v8
dataset_17403006_person_sql <- paste("
    SELECT
        person.person_id,
        person.birth_datetime as date_of_birth,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `person` person 
    LEFT JOIN
        `concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `cb_criteria` cr       
                        WHERE
                            concept_id IN (3023399, 37052205, 3006499, 3017044, 3044958, 4197602, 3045302, 3019762, 3032069, 3016806, 3038369, 3001019, 3016735, 3009201, 3019170, 4246879, 3015559, 40760528, 3009023, 3006939, 3027700)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `cb_criteria` cr       
                        WHERE
                            concept_id IN (35206909, 35206912, 35206914, 35206911, 35206913, 35206910, 1567993)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 0 
                        AND is_selectable = 1) 
                    AND is_standard = 0 )) criteria 
            UNION
            DISTINCT SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `cb_criteria` cr       
                        WHERE
                            concept_id IN (1567895, 1567893, 35206304)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 0 
                        AND is_selectable = 1) 
                    AND is_standard = 0 )) criteria 
            UNION
            DISTINCT SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `cb_criteria` cr       
                        WHERE
                            concept_id IN (44825214, 44829873, 44822938, 44827619, 44833373, 44820687, 44819497, 44828784, 44825267)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 0 
                        AND is_selectable = 1) 
                    AND is_standard = 0 )) criteria ) )", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
person_17403006_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "person_17403006",
  "person_17403006_*.csv")
message(str_glue('The data will be written to {person_17403006_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_17403006_person_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  person_17403006_path,
  destination_format = "CSV")



In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {person_17403006_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(sex_at_birth = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
demo_raw <- read_bq_export_from_workspace_bucket(person_17403006_path)

dim(demo_raw)

head(demo_raw, 5)

## related individuals

In [ ]:
system("gsutil -m cp -R ${WORKSPACE_BUCKET}/phenotype_files/ .")

In [ ]:
related = read.csv('phenotype_files/samples_relatedness_flagged_samples_v8.tsv')
head(related)

## GIA

In [ ]:
gia_pc = read.csv('phenotype_files/AOU.v8.GIA_PCA.txt', sep = '\t')
head(gia_pc)

# Clean Demographic Data

## Filter for Males and Females, Recode Sex (Male = 1, Female = 2), Change DOB to DateType

In [ ]:
sex_clean = demo_raw %>%
filter(grepl('Male|Female', sex_at_birth)) %>%
mutate(sex_at_birth = case_match(sex_at_birth,
                                 'Male' ~ 1,
                                 'Female' ~ 2)) %>%
mutate(date_of_birth = as.Date(date_of_birth))
head(sex_clean)
unique(sex_clean$sex_at_birth)

# Clean Lab Values for TSH Phenotyping
- Filter for TSH Measurements
- subset columns
- drop missing
- filter to outpatient
- filter to outpatient

In [ ]:
labs_raw %>%
filter(grepl('TSH|Thyroid|Thyrotropin', standard_concept_name)) %>%
filter(!grepl('T4|immunoglobulins|Ab|releasing|binding|2nd|15|30|--baseline', standard_concept_name)) %>%
group_by(standard_concept_name) %>%
summarize(n())

In [ ]:
tsh_labs = labs_raw %>%
filter(grepl('TSH|Thyroid|Thyrotropin',standard_concept_name)) %>%
filter(!grepl('T4|immunoglobulins|Ab|releasing|binding|2nd|15|30|--baseline', standard_concept_name)) %>%
select(person_id,
       measurement_datetime,
       value_as_number,
       unit_concept_name,
       visit_occurrence_concept_name) %>%
na.omit() %>%
filter(grepl('Outpatient Visit|Office Visit|Laboratory Visit', visit_occurrence_concept_name)) %>%
mutate(measurement_datetime = as.Date(measurement_datetime))
head(tsh_labs)

# Clean Pregnancy Data

## Rename columns

In [ ]:
preg_condition_raw = preg_condition_raw %>%
rename('date' = 'condition_start_datetime',
      'icd_code' = 'condition_source_value')
head(preg_condition_raw)

In [ ]:
preg_observation_raw = preg_observation_raw %>%
rename('date' = 'observation_datetime',
      'icd_code' = 'observation_source_value')
head(preg_observation_raw)

## Combine Condition and Observation Dataframes

In [ ]:
preg_all = rbind(preg_condition_raw, preg_observation_raw)
nrow(preg_condition_raw)
nrow(preg_observation_raw)
nrow(preg_all)

## Get Earliest Date

In [ ]:
preg_earliest_date = aggregate(x = preg_all$date, by = list(preg_all$person_id), FUN = min)
head(preg_earliest_date)
nrow(preg_earliest_date)

## Rename Columns & Change to Date Type

In [ ]:
preg_earliest_date = preg_earliest_date %>%
rename('person_id' = 'Group.1',
       'earliest_date_preg' = 'x') %>%
mutate(earliest_date_preg = as.Date(earliest_date_preg))
head(preg_earliest_date)

# Clean Medication Data

## Get Unique Individuals by Time of Their First Med

In [ ]:
meds_earliest_date = aggregate(x = meds_raw$drug_exposure_start_datetime, by = list(meds_raw$person_id), FUN = min)
head(meds_earliest_date)

## Rename Columns

In [ ]:
colnames(meds_earliest_date) = c('person_id', 'earliest_date_meds')
head(meds_earliest_date)

# Clean GIA/PC Data

## Change Research ID to Person ID, Subset to ancestry column

In [ ]:
gia_sub = gia_pc %>%
rename('person_id'='research_id') %>%
select(person_id, ancestry_pred)
head(gia_sub)

## Look at Ancestry Distribution

In [ ]:
gia_sub %>%
group_by(ancestry_pred) %>%
summarize(n())
nrow(gia_sub)

# Clean Related Individuals Data

## Rename ID Column

In [ ]:
colnames(related) = 'person_id'
head(related)

# Remove Excluding Measurements

## Pregnancy

### Merge Labs and Pregnancy Dataframes

In [ ]:
tsh_labs_preg = merge(tsh_labs, preg_earliest_date, by = 'person_id', all = T)
head(tsh_labs_preg)

### Make a Difftime Column (Pregnancy Time - TSH Time), Remove Rows with Negative Diff Time

In [ ]:
tsh_labs_no_preg_before = tsh_labs_preg %>%
mutate(difftime_preg_TSH = (difftime(earliest_date_preg, measurement_datetime, units = 'days'))) %>%
filter(!grepl('-', difftime_preg_TSH))

## Thyroid Meds

### Merge TSH/Preg Dataframe with Thyroid Meds Dataframe

In [ ]:
tsh_labs_no_preg_before_meds = merge(tsh_labs_no_preg_before,meds_earliest_date, by = 'person_id', all = T)
head(tsh_labs_no_preg_before_meds)

### Make a Difftime Column (Meds Time - TSH Time)

In [ ]:
tsh_labs_no_preg_before_meds_difftime = tsh_labs_no_preg_before_meds %>%
mutate(difftime_meds_TSH = (difftime(tsh_labs_no_preg_before_meds$earliest_date_meds,
                                     tsh_labs_no_preg_before_meds$measurement_datetime,
                                     units = 'days')))
head(tsh_labs_no_preg_before_meds_difftime)

### Remove Rows with Negative Diff Times (Meds First)

In [ ]:
tsh_labs_no_preg_meds_before = tsh_labs_no_preg_before_meds_difftime %>%
filter(!grepl('-', difftime_meds_TSH))
head(tsh_labs_no_preg_meds_before)
nrow(tsh_labs_no_preg_meds_before)

# Add Covariates

## Merge with TSH Labs Dataframe with Demographic Datafame

In [ ]:
tsh_labs_exclusions_demo = merge(tsh_labs_no_preg_meds_before, sex_clean, by = 'person_id')
head(tsh_labs_exclusions_demo)
nrow(tsh_labs_exclusions_demo)

## Calculate Age, Subset, Remove Missing, Filter to Adults, Rename Columns, Join with GIA

In [ ]:
tsh_labs_exclusions_demo_clean = tsh_labs_exclusions_demo %>%
mutate(age_days = (difftime(measurement_datetime, date_of_birth, units = 'days'))) %>%
mutate(age_days = gsub('days', '', age_days)) %>%
mutate(age_days = as.numeric(age_days)) %>%
mutate(AGE = age_days / 365) %>%
select(person_id, value_as_number, sex_at_birth, AGE) %>%
na.omit() %>%
filter(AGE >= 18) %>%
rename('TSH' = 'value_as_number',
       'SEX' = 'sex_at_birth') %>%
inner_join(gia_sub, by = 'person_id')
head(tsh_labs_exclusions_demo_clean)

In [ ]:
min(tsh_labs_exclusions_demo_clean$AGE, na.rm = T)
max(tsh_labs_exclusions_demo_clean$AGE, na.rm = T)

## Drop Related Individuals

In [ ]:
tsh_labs_exclusions_demo_clean = tsh_labs_exclusions_demo_clean[!(tsh_labs_exclusions_demo_clean$person_id %in% related$person_id),]
head(tsh_labs_exclusions_demo_clean)
nrow(tsh_labs_exclusions_demo_clean)

# Filter based on GIA

## Filter to Only Include EUR Individuals

In [ ]:
tsh_labs_exclusions_demo_eur = tsh_labs_exclusions_demo_clean %>%
filter(grepl('eur', ancestry_pred))
head(tsh_labs_exclusions_demo_eur)
nrow(tsh_labs_exclusions_demo_eur)

## Filter to Only Include AFR Individuals

In [ ]:
tsh_labs_exclusions_demo_afr = tsh_labs_exclusions_demo_clean %>%
filter(grepl('afr', ancestry_pred))
head(tsh_labs_exclusions_demo_afr)
nrow(tsh_labs_exclusions_demo_afr)

# Subset and Rename

## EUR

In [ ]:
tsh_labs_exclusions_demo_eur_sub = tsh_labs_exclusions_demo_eur %>%
select(-ancestry_pred) %>%
rename('IID' = 'person_id')
head(tsh_labs_exclusions_demo_eur_sub)

## AFR

In [ ]:
tsh_labs_exclusions_demo_afr_sub = tsh_labs_exclusions_demo_afr %>%
select(-ancestry_pred) %>%
rename('IID' = 'person_id')
head(tsh_labs_exclusions_demo_afr_sub)

# Take Mean TSH and Age & Remove Lab Values Not in Normal Range

## Take Mean

### EUR

In [ ]:
tsh_normal_range_eur = tsh_labs_exclusions_demo_eur_sub %>%
group_by(IID) %>% 
summarise_all(.funs = mean) %>%
filter(TSH <= 4.5 & TSH >= 0.45)
head(tsh_normal_range_eur)
nrow(tsh_normal_range_eur)
min(tsh_normal_range_eur$TSH)
max(tsh_normal_range_eur$TSH)

### AFR

In [ ]:
tsh_normal_range_afr = tsh_labs_exclusions_demo_afr_sub %>%
group_by(IID) %>% 
summarise_all(.funs = mean) %>%
filter(TSH <= 4.5 & TSH >= 0.45)
head(tsh_normal_range_afr)
nrow(tsh_normal_range_afr)
min(tsh_normal_range_afr$TSH)
max(tsh_normal_range_afr$TSH)

# Inverse Normal Transformation of Mean TSH, Take Age Squared

## EUR

In [ ]:
inv_norm_tsh_eur = tsh_normal_range_eur %>%
mutate(INV_NORMAL_TSH = qnorm((rank(TSH, na.last = "keep") - 0.5) / sum(!is.na(TSH)))) %>%
mutate(AGE_AGE = AGE^2) %>%
select(-c(TSH, AGE))
head(inv_norm_tsh_eur)

## AFR

In [ ]:
inv_norm_tsh_afr = tsh_normal_range_afr %>%
mutate(INV_NORMAL_TSH = qnorm((rank(TSH,na.last = "keep") - 0.5) / sum(!is.na(TSH)))) %>%
mutate(AGE_AGE = AGE^2) %>%
select(-c(TSH,AGE))
head(inv_norm_tsh_afr)

# Take EUR Random Sample for TSH

## get sample numbers

In [ ]:
nrow(inv_norm_tsh_eur)

In [ ]:
nrow(inv_norm_tsh_afr)

## take random sample

In [ ]:
set.seed(1234)

In [ ]:
eur_random_sample = data.frame(sample(x = inv_norm_tsh_eur$IID, size = 12385, replace = F))
colnames(eur_random_sample) = 'IID'
nrow(eur_random_sample)
head(eur_random_sample)

In [ ]:
inv_norm_tsh_eur_random = inv_norm_tsh_eur[inv_norm_tsh_eur$IID %in% eur_random_sample$IID,]
head(inv_norm_tsh_eur_random)
nrow(inv_norm_tsh_eur_random)

# Phenotype Free T4

## filter lab df by measurement name

In [ ]:
t4_names = labs_raw %>%
filter(grepl('T4' ,standard_concept_name)) %>%
filter(!grepl('TSH|Triiodothyronine|4th|albumin|Ab|total|index', standard_concept_name)) %>%
select(person_id, value_as_number, unit_source_value, unit_concept_name, measurement_datetime) %>%
na.omit()
length(unique(t4_names$person_id))
head(t4_names)

## make list of individuals with normal range TSH

In [ ]:
tsh_normal_range_indiv = inv_norm_tsh_eur %>%
rbind(inv_norm_tsh_afr) %>%
rbind(inv_norm_tsh_amr)
nrow(tsh_normal_range_indiv)

## filter individuals with t4 measurements to individuals w normal TSH

In [ ]:
t4_normal_range_tsh = t4_names[t4_names$person_id %in% tsh_normal_range_indiv$IID,]
length(unique(t4_normal_range_tsh$person_id))

## clean units based on Anne and Alisha's suggestions

Units selected:
- No matching concept
- nanogram per deciliter
- ng/dL
- picomole per liter

Unit specific transformation
- No matching concept: values > 0 & <= 2
- nanogram per deciliter: values > 0 & <= 2
- ng/dL: values > 0 & <= 2
- picomole per liter: divide by 12.872 and round to 2 digits


### filter to selected unit names

In [ ]:
t4_units = t4_normal_range_tsh%>%
filter(grepl('No matching concept|nanogram per deciliter|ng/dL|picomole per liter', unit_concept_name))
head(t4_units)
nrow(t4_units)

### filter to specific value ranges per unit name

In [ ]:
t4_No_matching_concept = t4_units%>%
filter(grepl('No matching concept', unit_concept_name)) %>%
filter(value_as_number > 0) %>%
filter(value_as_number <= 2)

nrow(t4_No_matching_concept)
min(t4_No_matching_concept$value_as_number)
max(t4_No_matching_concept$value_as_number)

In [ ]:
t4_nanogram_per_deciliter = t4_units %>%
filter(grepl('nanogram per deciliter',unit_concept_name)) %>%
filter(value_as_number > 0) %>%
filter(value_as_number <= 2)

head(t4_nanogram_per_deciliter)
nrow(t4_nanogram_per_deciliter)
min(t4_nanogram_per_deciliter$value_as_number)
max(t4_nanogram_per_deciliter$value_as_number)

In [ ]:
t4_ng_dL = t4_units %>%
filter(grepl('ng/dL', unit_concept_name)) %>%
filter(value_as_number > 0) %>%
filter(value_as_number <= 2)

nrow(t4_ng_dL)
min(t4_ng_dL$value_as_number)
max(t4_ng_dL$value_as_number)

In [ ]:
t4_picomole_per_liter = t4_units %>%
filter(grepl('picomole per liter', unit_concept_name)) %>%
mutate(value_as_number = value_as_number / 12.872) %>%
mutate(value_as_number = round(value_as_number, digits = 2))

nrow(t4_picomole_per_liter)
min(t4_picomole_per_liter$value_as_number)
max(t4_picomole_per_liter$value_as_number)

In [ ]:
t4_units_clean = t4_No_matching_concept %>%
rbind(t4_nanogram_per_deciliter,
      t4_ng_dL,
      t4_picomole_per_liter)
head(t4_units_clean)
nrow(t4_units_clean)

## merge with demo, calculate age, subset, and rename

In [ ]:
t4_demo = t4_units_clean %>%
merge(sex_clean, by = 'person_id') %>%
mutate(measurement_datetime = as.Date(measurement_datetime)) %>%
mutate(AGE = (difftime(measurement_datetime, date_of_birth, units = 'days'))) %>%
mutate(AGE = gsub('days', '', AGE)) %>%
mutate(AGE = as.numeric(AGE)) %>%
mutate(AGE = AGE / 365) %>%
select(person_id, value_as_number, sex_at_birth, AGE) %>%
na.omit() %>%
rename('FREE_T4' = 'value_as_number',
       'SEX' = 'sex_at_birth') %>%
inner_join(gia_sub, by = 'person_id') %>%
rename('IID' = 'person_id')
head(t4_demo)
nrow(t4_demo)

## split by ancestry

In [ ]:
t4_eur = t4_demo %>%
filter(ancestry_pred == 'eur') %>%
select(-c(ancestry_pred))
nrow(t4_eur)

In [ ]:
t4_afr = t4_demo %>%
filter(ancestry_pred == 'afr') %>%
select(-c(ancestry_pred))
nrow(t4_afr)

In [ ]:
t4_amr = t4_demo %>%
filter(ancestry_pred == 'amr') %>%
select(-c(ancestry_pred))
nrow(t4_amr)

## Take Mean Free T4 and Age

In [ ]:
t4_eur_mean = t4_eur %>%
group_by(IID) %>%
summarise_all(.funs = mean)
nrow(t4_eur_mean)

In [ ]:
t4_afr_mean = t4_afr %>%
group_by(IID) %>%
summarise_all(.funs = mean)
nrow(t4_afr_mean)

In [ ]:
t4_amr_mean = t4_amr %>%
group_by(IID) %>%
summarise_all(.funs = mean)
nrow(t4_amr_mean)

## Take Age Squared

In [ ]:
t4_eur_age_squared = t4_eur_mean %>%
mutate(AGE_AGE = AGE^2)
head(t4_eur_age_squared)

In [ ]:
t4_afr_age_squared = t4_afr_mean %>%
mutate(AGE_AGE = AGE^2)
head(t4_afr_age_squared)

In [ ]:
t4_amr_age_squared = t4_amr_mean %>%
mutate(AGE_AGE = AGE^2)
head(t4_amr_age_squared)

## Take EUR Random Sample

In [ ]:
t4_eur_random_sample = data.frame(sample(t4_eur_age_squared$IID, size = 4037, replace = F))
colnames(t4_eur_random_sample) = 'IID'
nrow(t4_eur_random_sample)

# Create, Concatenate, and Export Sample List

## Create Sample List

### TSH

In [ ]:
eur_sample_list = subset(inv_norm_tsh_eur, select = IID)
head(eur_sample_list)
nrow(eur_sample_list)

In [ ]:
eur_random_sample_list = eur_sample_list[eur_sample_list$IID %in% eur_random_sample$IID,]
nrow(eur_random_sample_list)
head(eur_random_sample_list)

In [ ]:
afr_sample_list = subset(inv_norm_tsh_afr, select = IID)
head(afr_sample_list)
nrow(afr_sample_list)

### Free T4

In [ ]:
eur_t4_sample_list = t4_eur_age_squared %>%
select(IID)
nrow(eur_t4_sample_list)

In [ ]:
eur_t4_random_sample_list = t4_eur_random_sample %>%
select(IID)
nrow(eur_t4_random_sample_list)

In [ ]:
afr_t4_sample_list = t4_afr_age_squared %>%
select(IID)
nrow(afr_t4_sample_list)

## Concatenate Sample Lists

In [ ]:
all_sample_lists = rbind(eur_sample_list, afr_sample_list, amr_sample_list)
head(all_sample_lists)
nrow(all_sample_lists)

In [ ]:
all_t4_sample_lists = rbind(eur_t4_sample_list, afr_t4_sample_list, amr_t4_sample_list)
nrow(all_t4_sample_lists)

## Export Sample Lists

### define workspace bucket location

In [ ]:
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

### TSH

#### All Ancestries

In [ ]:
write.table(all_sample_lists,
            'sample_lists/AoU_v8.TSH.GWAS.ALL.n=63859.sample_list.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

system(paste0("gsutil cp sample_lists/AoU_v8.TSH.GWAS.ALL.n=63859.sample_list.txt ", my_bucket, "/sample_lists/"), intern=T)

#### EUR

In [ ]:
write.table(eur_sample_list,
            'sample_lists/AoU_v8.TSH.GWAS.EUR.n=43057.sample_list.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

system(paste0("gsutil cp sample_lists/AoU_v8.TSH.GWAS.EUR.n=43057.sample_list.txt ", my_bucket, "/sample_lists/"), intern=T)

#### EUR Random Sample

In [ ]:
write.table(eur_random_sample_list,
            'sample_lists/AoU_v8.TSH.GWAS.EUR.n=12385.sample_list.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

system(paste0("gsutil cp sample_lists/AoU_v8.TSH.GWAS.EUR.n=12385.sample_list.txt ", my_bucket, "/sample_lists/"), intern=T)

#### AFR

In [ ]:
write.table(afr_sample_list,
            'sample_lists/AoU_v8.TSH.GWAS.AFR.n=12385.sample_list.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

system(paste0("gsutil cp sample_lists/AoU_v8.TSH.GWAS.AFR.n=12385.sample_list.txt ", my_bucket, "/sample_lists/"), intern=T)

### Free T4

#### All Ancestries

In [ ]:
write.table(all_t4_sample_lists,
            'AoU_v8.Free_T4.GWAS.ALL.n=20013.sample_list.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

system(paste0("gsutil cp sample_lists/AoU_v8.Free_T4.GWAS.ALL.n=20013.sample_list.txt ", " ", my_bucket, "/sample_lists/"), intern=T)

#### EUR

In [ ]:
write.table(eur_t4_sample_list,
            'AoU_v8.Free_T4.GWAS.EUR.n=13467.sample_list.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

system(paste0("gsutil cp sample_lists/AoU_v8.Free_T4.GWAS.EUR.n=13467.sample_list.txt ", " ", my_bucket, "/sample_lists/"), intern=T)

#### EUR random sample

In [ ]:
write.table(eur_t4_random_sample_list,
            'AoU_v8.Free_T4.GWAS.EUR.n=4037.sample_list.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

system(paste0("gsutil cp sample_lists/AoU_v8.Free_T4.GWAS.EUR.n=4037.sample_list.txt ", " ", my_bucket, "/sample_lists/"), intern=T)

#### AFR

In [ ]:
write.table(afr_t4_sample_list,
            'AoU_v8.Free_T4.GWAS.AFR.n=4037.sample_list.txt',
            sep = '\t',
            col.names = F,
            row.names = F,
            quote = F)

system(paste0("gsutil cp sample_lists/AoU_v8.Free_T4.GWAS.AFR.n=4037.sample_list.txt ", " ", my_bucket, "/sample_lists/"), intern=T)

# Read in Ancestry Specific PCs

## EUR

### TSH

#### eigenvec

In [ ]:
eur_tsh_eigenvec = read.csv('EUR.TSH.PCA_cleaned.eigenvec', header = F)
nrow(eur_tsh_eigenvec)
head(eur_tsh_eigenvec)

### eigenval

In [ ]:
eur_tsh_eigenval = read.csv('EUR.TSH.PCA_cleaned.eigenval', header = F)
head(eur_tsh_eigenval)

### Free T4

#### eigenvec

In [ ]:
eur_t4_eigenvec = read.csv('EUR.Free_T4.PCA_cleaned.eigenvec', header = F)
nrow(eur_t4_eigenvec)
head(eur_t4_eigenvec)

#### eigenval

In [ ]:
eur_t4_eigenval = read.csv('EUR.Free_T4.PCA_cleaned.eigenval', header = F)
head(eur_t4_eigenval)

## AFR

### TSH

#### eigenvec

In [ ]:
afr_tsh_eigenvec = read.csv('AFR.TSH.PCA_cleaned.eigenvec', header = F)
nrow(afr_tsh_eigenvec)
head(afr_tsh_eigenvec)

#### eigenval

In [ ]:
afr_tsh_eigenval = read.csv('AFR.TSH.PCA_cleaned.eigenval', header = F)
head(afr_tsh_eigenval)

### Free T4

#### eigenvec

In [ ]:
afr_t4_eigenvec = read.csv('AFR.Free_T4.PCA_cleaned.eigenvec', header = F)
nrow(afr_t4_eigenvec)
head(afr_t4_eigenvec)

#### eigenval

In [ ]:
afr_t4_eigenval = read.csv('AFR.Free_T4.PCA_cleaned.eigenval', header = F)
head(afr_t4_eigenval)

# Clean Ancestry Specific PCs Data

## Clean Eigenvals
- add PC column
- make PC column numeric
- calculate variance

### TSH

In [ ]:
eur_tsh_eigenval_clean = eur_tsh_eigenval %>%
mutate(PC = rownames(eur_tsh_eigenval)) %>%
mutate(PC = as.numeric(PC)) %>%
mutate(VARIANCE = V1 / sum(V1))
head(eur_tsh_eigenval_clean)

In [ ]:
afr_tsh_eigenval_clean = afr_tsh_eigenval %>%
mutate(PC = rownames(afr_tsh_eigenval)) %>%
mutate(PC = as.numeric(PC)) %>%
mutate(VARIANCE = V1 / sum(V1))
head(afr_tsh_eigenval_clean)

### Free T4

In [ ]:
eur_t4_eigenval_clean = eur_t4_eigenval %>%
mutate(PC = rownames(eur_t4_eigenval)) %>%
mutate(PC = as.numeric(PC)) %>%
mutate(VARIANCE = V1 / sum(V1))
head(eur_t4_eigenval_clean)

In [ ]:
afr_t4_eigenval_clean = afr_t4_eigenval %>%
mutate(PC = rownames(afr_t4_eigenval)) %>%
mutate(PC = as.numeric(PC)) %>%
mutate(VARIANCE = V1 / sum(V1))
head(afr_t4_eigenval_clean)

## Clean Eigenvectors

### TSH

In [ ]:
eur_tsh_eigenvec_clean = eur_tsh_eigenvec %>%
rename('IID' = 'V1',
       'PC1' = 'V2',
       'PC2' = 'V3',
       'PC3' = 'V4',
       'PC4' = 'V5') %>%
select(IID, PC1, PC2, PC3, PC4)
head(eur_tsh_eigenvec_clean)

In [ ]:
afr_tsh_eigenvec_clean = afr_tsh_eigenvec %>%
rename('IID' = 'V1',
       'PC1' = 'V2') %>%
select(IID, PC1)
head(afr_tsh_eigenvec_clean)

### Free T4

In [ ]:
eur_t4_eigenvec_clean = eur_t4_eigenvec %>%
rename('IID' = 'V1',
       'PC1' = 'V2',
       'PC2' = 'V3',
       'PC3' = 'V4',
       'PC4' = 'V5') %>%
select(IID, PC1, PC2, PC3, PC4)
head(eur_t4_eigenvec_clean)

In [ ]:
afr_t4_eigenvec_clean = afr_t4_eigenvec %>%
rename('IID' = 'V1',
       'PC1' = 'V2') %>%
select(IID, PC1)
head(afr_t4_eigenvec_clean)

# Make Scree Plots

## TSH

### EUR

In [ ]:
ggplot(eur_tsh_eigenval_clean, aes(x = PC, y = VARIANCE)) +
geom_line() +
scale_x_continuous(breaks = seq(1, 20, by = 1), limits = c(1, 20)) +
ggtitle('AOU v8 EUR TSH Scree Plot')
ggsave('plots/AoU_v8.TSH.PCA.EUR.n=12385.scree_plot.png',
       width = 10,
       height = 10,
       dpi = 300)

### AFR

In [ ]:
ggplot(afr_tsh_eigenval_clean, aes(x = PC, y = VARIANCE)) +
geom_line() +
scale_x_continuous(breaks = seq(1, 20, by = 1), limits = c(1, 20)) +
ggtitle('AOU v8 AFR TSH Scree Plot')
ggsave('plots/AoU_v8.TSH.PCA.AFR.n=12385.scree_plot.png',
       width = 10,
       height = 10,
       dpi = 300)

## Free T4

### EUR

In [ ]:
ggplot(eur_t4_eigenval_clean, aes(x = PC, y = VARIANCE)) +
geom_line() +
scale_x_continuous(breaks = seq(1, 20, by = 1), limits = c(1, 20)) +
ggtitle('AOU v8 EUR Free T4 Scree Plot')
ggsave('plots/AoU_v8.Free_T4.PCA.EUR.n=4037.scree_plot.png',
       width = 10,
       height = 10,
       dpi = 300)

### AFR

In [ ]:
ggplot(afr_t4_eigenval_clean, aes(x = PC, y = VARIANCE)) +
geom_line() +
scale_x_continuous(breaks = seq(1, 20, by = 1), limits = c(1, 20)) +
ggtitle('AOU v8 AFR Free T4 Scree Plot')
ggsave('plots/AoU_v8.Free_T4.PCA.AFR.n=4037.scree_plot.png',
       width = 10,
       height = 10,
       dpi = 300)

# Merge Dataframe with Ancestry Specific PC Dataframe

## TSH

### EUR

In [ ]:
inv_norm_tsh_eur_random_pc = inv_norm_tsh_eur %>%
merge(eur_tsh_eigenvec_clean, by = 'IID')
head(inv_norm_tsh_eur_random_pc)
nrow(inv_norm_tsh_eur_random_pc)

### AFR

In [ ]:
inv_norm_tsh_afr_pc = inv_norm_tsh_afr %>%
merge(afr_tsh_eigenvec_clean, by = 'IID')
head(inv_norm_tsh_afr_pc)
nrow(inv_norm_tsh_afr_pc)

## Free T4

### EUR

In [ ]:
t4_eur_pc = t4_eur_age_squared %>%
merge(eur_t4_eigenvec_clean, by = 'IID')
head(t4_eur_pc)
nrow(t4_eur_pc)

### AFR

In [ ]:
t4_afr_pc = t4_afr_age_squared %>%
merge(afr_t4_eigenvec_clean, by = 'IID')
head(t4_afr_pc)
nrow(t4_afr_pc)

# Assess AMR Covariate Correlation

In [ ]:
corrplot(cor(inv_norm_tsh_age_squared_amr_pc_sub),method='color',type = 'lower')

In [ ]:
corrplot(cor(inv_norm_tsh_age_squared_amr_pc_sub),method='number',type = 'lower')

# Export Phenotype Covariate Files

In [ ]:
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

## TSH

### EUR random sample

In [ ]:
write.table(inv_norm_tsh_eur_random_pc,
            'phenotype_files/AoU_v8.TSH.GWAS.EUR.n=12385.phenotype_covariates.txt',
            sep = '\t',
            col.names = T,
            row.names = F,
            quote = F)

system(paste0("gsutil cp phenotype_files/AoU_v8.TSH.GWAS.EUR.n=12385.phenotype_covariates.txt ", my_bucket, "/phenotype_files/"), intern=T)

### AFR

In [ ]:
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')
# store the dataframe in current workspace
write.table(inv_norm_tsh_afr_pc,
            'AoU_v8.TSH.GWAS.AFR.n=12385.phenotype_covariates.txt',
            sep = '\t',
            col.names = T,
            row.names = F,
            quote = F)

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp AoU_v8.TSH.GWAS.AFR.n=12385.phenotype_covariates.txt ", my_bucket, "/phenotype_files/"))

## Free T4

### EUR- random sample

In [ ]:
write.table(t4_eur_pc,
            'phenotype_files/AoU_v8.Free_T4.GWAS.EUR.n=4037.phenotype_covariates.txt',
            sep = '\t',
            col.names = T,
            row.names = F,
            quote = F)

system(paste0("gsutil cp phenotype_files/AoU_v8.Free_T4.GWAS.EUR.n=4037.phenotype_covariates.txt ", my_bucket, "/phenotype_files/"), intern=T)

### AFR

In [ ]:
write.table(t4_afr_pc,
            'phenotype_files/AoU_v8.Free_T4.GWAS.AFR.n=4037.phenotype_covariates.txt',
            sep = '\t',
            col.names = T,
            row.names = F,
            quote = F)

system(paste0("gsutil cp phenotype_files/AoU_v8.Free_T4.GWAS.AFR.n=4037.phenotype_covariates.txt ", my_bucket, "/phenotype_files/"), intern=T)